In [4]:
import pandas as pd
import numpy as np
import jieba
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [5]:
df = pd.read_csv("course_data.csv")
print("課程資料預覽:")
display(df.head())

# 補缺失值
for col in ['課程名稱', '課程概述']:
    if col in df.columns:
        df[col] = df[col].fillna("")
    else:
        raise ValueError(f"找不到 '{col}' 欄位，請檢查 CSV 格式。")

# 將課程名稱與描述合併
df['combined_text'] = df['課程名稱'] + " " + df['課程概述']


課程資料預覽:


,課程代碼,系所名稱,課程名稱,授課教師,課程類型,學分數,上課地點,課程概述,Unnamed: 8
0,3001,通識課程:人文領域,人文：生命美學─藝術,王雪萍,必修課,0-2,芳華廳,人文領域的課程目標在於啟發學生美的鑑賞與願意為善的藝術涵養，本課程將透過西洋古典音樂來達成此...,NaN
1,3002,通識課程:人文領域,人文：生命美學─藝術,蔡永凱,必修課,0-2,芳華廳,本課程前身為「人文：悠遊聲樂與鋼琴的世界」，訴求藉由古典音樂之欣賞，探索以下議題：\r\n1...,NaN
2,3003,通識課程:人文領域,人文：生命美學─藝術,林彥良,必修課,0-2,H207,從美的探討與藝術的起源開始，到八大藝術以及東西洋美術作品的介紹與欣賞，運用ＰＰＴ檔與相關複製...,NaN
3,3004,通識課程:人文領域,人文：生命美學─藝術,沈裕昌,必修課,0-2,HT101,本課程將帶領修課同學觀賞日本、韓國、越南、泰國等亞洲諸國電影導演的經典作品，並與台灣電影進行...,NaN
4,3005,通識課程:人文領域,人文：生命美學─藝術,張文鴻,必修課,0-2,C106,(一)人文:生命美學的課程目標在於啟迪學生美的鑑賞與願意為善的藝術涵養。\r\n(二)生命繪...,NaN


In [6]:
def chinese_tokenizer(text):
    return list(jieba.cut(text))

df['combined_text'] = df['combined_text'].apply(lambda x: " ".join(chinese_tokenizer(x)))
print("斷詞後文本預覽:")
display(df['combined_text'].head())


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/dv/mvx0p9tn1bv13m957fnhyynw0000gn/T/jieba.cache
Loading model cost 0.784 seconds.
Prefix dict has been built successfully.


斷詞後文本預覽:


0    人文 ： 生命 美學 ─ 藝術   人文 領域 的 課程 目標 在 於 啟發學生 美的 鑑賞...
1    人文 ： 生命 美學 ─ 藝術   本 課程 前身 為 「 人文 ： 悠遊 聲樂 與 鋼琴 ...
2    人文 ： 生命 美學 ─ 藝術   從 美的 探討 與 藝術 的 起源 開始 ， 到 八大 ...
3    人文 ： 生命 美學 ─ 藝術   本 課程 將帶 領修 課同學 觀賞 日本 、 韓國 、 ...
4    人文 ： 生命 美學 ─ 藝術   ( 一 ) 人文 : 生命 美學 的 課程 目標 在 於...
Name: combined_text, dtype: object

In [7]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])
print("TF-IDF 矩陣形狀:", tfidf_matrix.shape)


TF-IDF 矩陣形狀: (694, 8697)


In [8]:
tf_idf_weights = np.ones(len(df))  # 初始權重為 1.0
ratings = {}  # 存儲課程評分

def recommend_courses(query, top_n=5):
    query_processed = " ".join(chinese_tokenizer(query))
    query_vec = vectorizer.transform([query_processed])
    cosine_sim = cosine_similarity(query_vec, tfidf_matrix).flatten()
    weighted_sim = cosine_sim * tf_idf_weights  # 根據評分調整權重
    top_indices = weighted_sim.argsort()[::-1][:top_n]
    recommendations = df.iloc[top_indices].copy()
    recommendations['similarity'] = weighted_sim[top_indices]
    return recommendations[['課程代碼','課程名稱', '課程概述', 'similarity']]


In [9]:
# 互動式查詢
query_text = widgets.Text(
    value='請推薦排球課程給我',
    placeholder='在此輸入查詢...',
    description='查詢:',
    disabled=False
)
search_button = widgets.Button(description='查詢')
output_area = widgets.Output()


In [10]:
def recommend_courses(query, top_n=5):
    query_processed = " ".join(chinese_tokenizer(query))
    query_vec = vectorizer.transform([query_processed])
    cosine_sim = cosine_similarity(query_vec, tfidf_matrix).flatten()
    weighted_sim = cosine_sim * tf_idf_weights  # 使用權重調整相似度
    top_indices = weighted_sim.argsort()[::-1][:top_n]
    recommendations = df.iloc[top_indices].copy()
    recommendations['similarity'] = weighted_sim[top_indices]
    recommendations['index'] = top_indices  # 保留原始索引以利後續更新權重
    return recommendations[['index','課程代碼','課程名稱', '課程概述', 'similarity']]

def get_rating_widget(course_index):
    rating_widget = widgets.Dropdown(
        options=[(str(i), i) for i in range(1, 6)],
        description=f'課程 {course_index+1} 評分:',
        disabled=False
    )
    return rating_widget

rating_widgets = []


def on_search_button_clicked(b):
    with output_area:
        clear_output()
        query = query_text.value
        print(f"查詢內容: {query}\n")
        results = recommend_courses(query, top_n=5)
        if results.empty:
            print("找不到符合的課程。")
        else:
            print("推薦結果:")
            display(results)
            global rating_widgets
            rating_widgets = [get_rating_widget(i) for i in range(len(results))]
            display(*rating_widgets, rate_button)

search_button.on_click(on_search_button_clicked)


In [11]:
query_text = widgets.Text(
    value='請推薦排球課程給我',
    placeholder='在此輸入查詢...',
    description='查詢:',
    disabled=False
)
search_button = widgets.Button(description='查詢')
output_area = widgets.Output()

# 評分按鈕
rate_button = widgets.Button(description='提交評分')

# 用來存放每筆推薦結果的評分 widget
rating_widgets = {}

def display_recommendations_with_rating(results):
    # 依每筆推薦結果建立一個水平容器(HBox)，包含課程資訊與評分框
    rows = []
    for idx, row in results.iterrows():
        # 建立評分下拉選單
        rating_widget = widgets.Dropdown(
            options=[(str(i), i) for i in range(1, 6)],
            description='評分:',
            value=5  # 預設給最高評分，也可調整預設值
        )
        rating_widgets[int(row['index'])] = rating_widget
        # 建立課程資訊文字
        course_info = f"{row['課程代碼']} | {row['課程名稱']} |  {row['課程概述']} | 相似度: {row['similarity']:.4f}"
        info_widget = widgets.Label(value=course_info)
        # 將課程資訊與評分 widget 放在同一行
        row_box = widgets.HBox([info_widget, rating_widget])
        rows.append(row_box)
    display(*rows, rate_button)

def on_search_button_clicked(b):
    with output_area:
        clear_output()
        query = query_text.value
        print(f"查詢內容: {query}\\n")
        results = recommend_courses(query, top_n=5)
        if results.empty:
            print("找不到符合的課程。")
        else:
            print("推薦結果：")
            display_recommendations_with_rating(results)

def on_rate_button_clicked(b):
    global ratings, tf_idf_weights
    # 根據評分 widget 的值更新評分與權重
    for idx, widget_item in rating_widgets.items():
        score = widget_item.value
        course_name = df.iloc[idx]['課程名稱']
        ratings[course_name] = score
        # 更新該筆資料的權重：權重 = 1 + (評分 / 5)
        tf_idf_weights[idx] = 1 + (score / 5)
    with output_area:
        print("評分已提交，系統將根據評分優化推薦結果！")

search_button.on_click(on_search_button_clicked)
rate_button.on_click(on_rate_button_clicked)

# 顯示查詢介面
display(query_text, search_button, output_area)

Text(value='請推薦排球課程給我', description='查詢:', placeholder='在此輸入查詢...')

Button(description='查詢', style=ButtonStyle())

Output()

如果你不需要評分 可以看這下面的程式

In [12]:
def on_search_button_clicked(b):
    with output_area:
        clear_output()  # 清除上一次的輸出
        query = query_text.value
        print(f"查詢內容: {query}\n")
        results = recommend_courses(query, top_n=5)
        if results.empty:
            print("找不到符合的課程。")
        else:
            print("推薦結果:")
            display(results)

search_button.on_click(on_search_button_clicked)

# 顯示互動介面
display(query_text, search_button, output_area)


Text(value='請推薦排球課程給我', description='查詢:', placeholder='在此輸入查詢...')

Button(description='查詢', style=ButtonStyle())

Output()